In [149]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [150]:
input = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]],  dtype='float32')

In [151]:
target = np.array([[56], 
                    [81], 
                    [119], 
                    [22], 
                    [103]],  dtype='float32')

In [153]:
input = torch.from_numpy(input).float()
target = torch.from_numpy(target).float()
print(input.type())

torch.FloatTensor


In [154]:
w=torch.randn(1,3,requires_grad=True)
b=torch.randn(1,requires_grad=True)

In [155]:
def model(x):
    return x @ w.t() + b

In [156]:
pred = model(input)

In [157]:
def mse(pred,target):
    diff = pred-target
    return torch.sum(diff*diff)/diff.numel()

In [158]:
loss = mse(pred,target)

In [159]:
loss.backward()

In [160]:
print(w.grad)

tensor([[-40308.0078, -45698.4180, -27718.2852]])


In [161]:
for i in range(1000):
    pred = model(input)
    loss = mse(pred,target)
    loss.backward()
    with torch.no_grad():
        w -= w.grad*1e-5
        b -= b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [162]:
print(loss)

tensor(1.1069, grad_fn=<DivBackward0>)


In [163]:
pred

tensor([[ 57.1002],
        [ 82.0067],
        [119.2509],
        [ 21.2457],
        [101.3633]], grad_fn=<AddBackward0>)

In [164]:
target

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

In [165]:
# Using torch.nn

In [166]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [168]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [169]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size , shuffle=True)

In [170]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[101.,  44.,  37.],
        [ 91.,  87.,  65.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])
tensor([[ 21.,  38.],
        [ 80., 102.],
        [ 56.,  70.],
        [ 20.,  38.],
        [102., 120.]])


In [171]:
model = nn.Linear(3,2)
list(model.parameters())

[Parameter containing:
 tensor([[-0.4743, -0.3238,  0.5498],
         [ 0.1398, -0.3386,  0.3974]], requires_grad=True),
 Parameter containing:
 tensor([-0.2536, -0.2634], requires_grad=True)]

In [174]:
pred=model(inputs)

In [175]:
loss_fn = nn.functional.mse_loss

In [176]:
loss = loss_fn(pred, targets)

In [177]:
print(loss)

tensor(11866.8340, grad_fn=<MseLossBackward>)


In [189]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-6)

In [190]:
def fit(epoch_no, model, loss_fn, opt, train_dl):
    
    for epoch in range(epoch_no):
        for xb,yb in train_dl:
            
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            
            opt.step()
            opt.zero_grad()
            
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epoch_no, loss.item()))

In [191]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 0.9182
Epoch [20/1000], Loss: 0.8446
Epoch [30/1000], Loss: 0.9577
Epoch [40/1000], Loss: 0.8119
Epoch [50/1000], Loss: 1.2637
Epoch [60/1000], Loss: 1.4697
Epoch [70/1000], Loss: 0.9000
Epoch [80/1000], Loss: 0.7439
Epoch [90/1000], Loss: 1.0726
Epoch [100/1000], Loss: 0.7693
Epoch [110/1000], Loss: 1.3669
Epoch [120/1000], Loss: 1.1877
Epoch [130/1000], Loss: 1.4509
Epoch [140/1000], Loss: 1.1438
Epoch [150/1000], Loss: 1.0834
Epoch [160/1000], Loss: 0.9274
Epoch [170/1000], Loss: 1.0044
Epoch [180/1000], Loss: 0.8654
Epoch [190/1000], Loss: 0.8720
Epoch [200/1000], Loss: 0.9678
Epoch [210/1000], Loss: 1.0087
Epoch [220/1000], Loss: 0.8976
Epoch [230/1000], Loss: 1.3037
Epoch [240/1000], Loss: 0.9210
Epoch [250/1000], Loss: 0.9693
Epoch [260/1000], Loss: 0.8637
Epoch [270/1000], Loss: 1.1144
Epoch [280/1000], Loss: 1.4755
Epoch [290/1000], Loss: 0.8425
Epoch [300/1000], Loss: 0.9088
Epoch [310/1000], Loss: 1.0031
Epoch [320/1000], Loss: 0.7653
Epoch [330/1000],

In [193]:
x = model(torch.tensor([100., 100., 100.]))
x

tensor([114.0549, 139.4468], grad_fn=<AddBackward0>)